<a href="https://colab.research.google.com/github/younglaecho/learn_machine_learning/blob/main/chap05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 교차검증과 그리드 서치


모델을 만들어가는 과정에서 우리는 여러 형태의 모델을 훈련하고 테스트하게 된다.<br>
이 때, 우리가 test 세트를 사용해서 계속 성능을 확인하다보면 점점 테스트 세트에 맞추어진 모델을 만들게 된다.<br>
이를 방지하기 위해 훈련세트에서 따로 일부를 때어내어 검증세트라는 것을 만들어 사용하게 된다.

## 검증 세트

In [1]:
import pandas as pd
wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [2]:
data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

In [3]:
from sklearn.model_selection import train_test_split
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42) 

검증세트를 만드겠습니다.

In [4]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

In [5]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [6]:
from sklearn.tree import DecisionTreeClassifier
dt = DecisionTreeClassifier()
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.8596153846153847


## 교차 검증

안정적인 검증점수를 얻기 위해 교차검증이란 것을 합니다.<br>
교차검증은 검증세트를 떼어내어 평가하는 과정을 여러번 반복하는 것을 말합니다.<br>
여러 번 한 것을 평균하여 최종 검증 점수를 얻습니다.<br>
일반적으로 5-폴드, 10-폴드를 많이 사용한다고 합니다. 이렇게 하면 데이터의 80~90%까지를 훈련에 사용할 수 있습니다.

cross_validate()라는 교차 검증 함수를 활용해보겠습니다.


In [7]:
from sklearn.model_selection import cross_validate
scores = cross_validate(dt, train_input, train_target)
print(scores)
                      

{'fit_time': array([0.00686264, 0.00589013, 0.00643945, 0.00636458, 0.00589204]), 'score_time': array([0.00068092, 0.00050712, 0.00052786, 0.0005486 , 0.00050926]), 'test_score': array([0.87211538, 0.85      , 0.87872955, 0.85274302, 0.83926853])}


기본 값으로 5-폴드가 실행됩니다.<br>
* fit_time: fit하는데 걸린 시간<br>
* score_time: score를 내는데 걸린 시간<br>
* test_score: score<br>

아래에서 교차 검증을 최종 점수를 구하겠습니다.


In [8]:
import numpy as np

print(np.mean(scores['test_score']))

0.858571296364848


현재 데이터에서는 우리가 train_test_split을 사용해 훈련 셋을 섞어 두었지만, <br>
교차 검증에서 데이터를 섞어 줄 필요가 있다면 splitter 라는 것을 사용해야합니다.

회귀 모델 : KFold 분할기<br>
분류 모델 : StratifiedKFold 분할기

In [9]:
from sklearn.model_selection import StratifiedKFold
scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.8587637891463686


아래는 10-폴드 교차 검증입니다.

In [10]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.860114865866311


### 하이퍼 파라미터 튜닝

하이퍼 파라미터는 모델마다 적게는 1 ~ 2개, 많게는 5 ~ 6개가 있습니다.<br>
문제는 하나의 값을 최적의 값으로 고정하고, 다른 값을 찾아내는 방식이 가능하지 않다는 것입니다.<br>
매개변수의 양이 많아질 수록 직접 찾아내는 것이 어려워집니다.<br>
이를 해결하기 위해 사이킷런에서는 그리드 서치라는 것을 제공합니다.

결정트리 알고리즘의 min_impurity_decrease 파라미터를 예시로 구현해보겠습니다.

In [11]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

GridSearchCV의 매개변수 기본값 5입니다. 즉 5-폴드 교차 검증을 수행합니다.<br>
즉 5x5개의 모델을 훈련합니다. n_jobs = -1로 설정하면 시스템의 모든 코어를 사용한다. 

In [12]:
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

best_estimator_에 25개의 모델 중에서 가장 점수가 높을 모델으로 전체 train 세트를 학습한 모델이 들어갑니다.


In [13]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.9615162593804117


가장 점수가 높게 나온 하이퍼파라미터가 best_params_에 담깁니다.

In [14]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


cv_results_를 통해 전체 score를 확인할수 있다.

In [17]:
print(gs.cv_results_)
print(gs.cv_results_['mean_test_score'])

{'mean_fit_time': array([0.00847259, 0.00746751, 0.00698519, 0.00909309, 0.00623603]), 'std_fit_time': array([0.00062805, 0.00013216, 0.00010415, 0.00234073, 0.00032025]), 'mean_score_time': array([0.00088024, 0.00084167, 0.00084906, 0.00214186, 0.00076518]), 'std_score_time': array([3.46060276e-05, 8.81850218e-05, 8.59641521e-05, 2.14569304e-03,
       1.25472212e-04]), 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
             mask=[False, False, False, False, False],
       fill_value='?',
            dtype=object), 'params': [{'min_impurity_decrease': 0.0001}, {'min_impurity_decrease': 0.0002}, {'min_impurity_decrease': 0.0003}, {'min_impurity_decrease': 0.0004}, {'min_impurity_decrease': 0.0005}], 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]), 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]), 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216,

In [16]:
best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


여러 개의 하이퍼 파라미터를 동시에 그리드 서치 해보겠습니다.

In [18]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20, 1),
          'min_samples_split': range(2, 100, 10)}

이대로 실행하면 9x15x10 = 1350 번의 교차검증을 수행하게 되고<br>
5-폴드 교차검증을 수행하므로 모델의 수는 6750개가 됩니다.

In [20]:
gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

최상의 매개변수 조합을 확인해보겠습니다.

In [22]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


최대 교차 검증 점수도 확인해보겠습니다.

In [23]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8683865773302731


## 랜덤 서치

매개변수의 값이 수치일 때 값의 범위나 간격을 미리 정하기 어려울 수 있습니다. 또한 너무 많은 매개변수 조건이 있어 그리드 서치 수행시간이 오래걸릴 수 있습니다. 이를 위해 랜덤 서치를 활용해보겠습니다.

scipy의 uniform과 randint 를 활용하여 구현해보겠습니다.

In [30]:
from scipy.stats import uniform, randint
rgen = randint(0, 10)
print(type(rgen.rvs(10)))
print(rgen.rvs(10))

<class 'numpy.ndarray'>
[5 6 3 9 5 9 0 6 5 3]


In [31]:
np.unique(rgen.rvs(1000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 81, 110, 106, 106, 100,  85, 102, 101, 109, 100]))

randint 객체를 만들고 rvs라는 메서드를 사용하여 범위내의 정수를 고르게 뽑아낸다.


uniform도 똑같은 방식으로 사용할 수 있다.

In [32]:
ugen = uniform(0,1)
ugen.rvs(10)

array([0.0462902 , 0.73140802, 0.5993152 , 0.59226129, 0.54830856,
       0.13330111, 0.89771868, 0.52552833, 0.10886974, 0.59606008])

이를 활용하여 랜덤서치를 통해 예측해보겠습니다.

In [33]:
params = {'min_impurity_decrease': uniform(0.001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2,25),
          'min_samples_leaf': randint(1,25)}


min_impurity_decrease : 불순도 감소 최소량<br>
max_depth 
- default = None
- 최대 깊이 
- 완벽하게 클래스 값이 결정될 때 까지 분할
또는 데이터 개수가 min_samples_split보다 작아질 때까지 분할
- 깊이가 깊어지면 과적합될 수 있으므로 적절히 제어 필요

min_samples_split 
- 노드를 분할하기 위한 최소한의 샘플 데이터수 → 과적합을 제어하는데 사용
- Default = 2 → 작게 설정할 수록 분할 노드가 많아져 과적합 가능성 증가

min_samples_leaf 
- 리프노드가 되기 위해 필요한 최소한의 샘플 데이터수
- min_samples_split과 함께 과적합 제어 용도
- 불균형 데이터의 경우 특정 클래스의 데이터가 극도로 작을 수 있으므로 작게 설정 필요


In [48]:
from sklearn.model_selection import RandomizedSearchCV
gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42, splitter='random'), params, n_iter=100, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42,
                                                    splitter='random'),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb37d77fa90>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb37d80e690>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb37d770950>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_frozen object at 0x7fb37d77f290>},
                   random_state=42)

params에는 난수생성기(?)객체를 넣고, 랜덤서치 클래스인 RandomizedSearchCV의 매개변수로 n_iter(조합의 수)를 입력한다.

In [49]:
print(gs.best_params_)

{'max_depth': 24, 'min_impurity_decrease': 0.0012301852682415552, 'min_samples_leaf': 16, 'min_samples_split': 20}


In [50]:
print(np.max(gs.cv_results_['mean_test_score']))

0.81778003997927


In [51]:
Dt = gs.best_estimator_
print(dt.score(test_input, test_target))

0.8653846153846154
